In [2]:
print('ok')

ok


In [3]:
!nvidia-smi

Sun Feb 25 06:35:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [5]:
# accelerate is a library for easy GPU/TPU training with PyTorch and TensorFlow.
# transformers library provides state-of-the-art pre-trained models for natural language processing tasks
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
  Using cached transformers-4.38.1-py3-none-any.whl (8.5 MB)
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [6]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

# own translation system
# language translation system
# from the hugging face we are going to use open source translation model
# data set also i wll take from hugging face


# Transfer learning
# FineTuning(for this translaiton system we gonnda perform finetune of pretrain model)
# RAG System(will implement it in my future class-> as of now dont focous here)

Google- T5

Facebook- M2M100

Helinsinki-NLP

Google Cloud Translaiton API

Tune AI

Amazone Comprehend

In [7]:
model="Helsinki-NLP/opus-mt-en-hi"

In [8]:
dataset=load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [10]:
dataset["train"]

Dataset({
    features: ['translation'],
    num_rows: 1659083
})

In [11]:
dataset["train"][0]

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

In [12]:
dataset["train"][1]

{'translation': {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}}

In [13]:
dataset["validation"]

Dataset({
    features: ['translation'],
    num_rows: 520
})

In [14]:
dataset["validation"][0]

{'translation': {'en': 'Students of the Dattatreya city Municipal corporation secondary school demonstrated their imagination power by creating the fictitious fort "Duttgarh".',
  'hi': "महानगर पालिका अंतर्गत दत्तात्रय नगर माध्यमिक स्कूल के विद्यार्थियों ने काल्पनिक किला 'दत्तगढ़' बनाकर अपनी कल्पनाशक्ति का परिचय दिया।"}}

In [15]:
dataset["test"]

Dataset({
    features: ['translation'],
    num_rows: 2507
})

In [16]:
dataset["test"][0]

{'translation': {'en': 'A black box in your car?',
  'hi': 'आपकी कार में ब्लैक बॉक्स?'}}

In [17]:
model

'Helsinki-NLP/opus-mt-en-hi'

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [19]:
tokenizer("there is sunny who is teachine data science")

{'input_ids': [188, 23, 41670, 66, 23, 2144, 6131, 1195, 7373, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
tokenizer(["hello this is sunny", "this is another sunny"])

{'input_ids': [[39915, 90, 23, 41670, 0], [90, 23, 414, 41670, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [21]:
tokenizer(['चाय पार्टी भौचक्की है।'])

{'input_ids': [[44, 4228, 260, 1321, 44, 2703, 260, 428, 1185, 2136, 130, 44, 1, 1185, 716, 130, 44, 1, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [22]:
with tokenizer.as_target_tokenizer():
  print(tokenizer(['चाय पार्टी भौचक्की है।']))

{'input_ids': [[15889, 9384, 61313, 130, 5, 40, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [24]:
dataset["train"]["translation"]

[{'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'},
 {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'},
 {'en': 'The default plugin layout for the bottom panel',
  'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'The default plugin layout for the top panel',
  'hi': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'A list of plugins that are disabled by default',
  'hi': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है'},
 {'en': 'Highlight duration', 'hi': 'अवधि को हाइलाइट रकें'},
 {'en': 'The duration of the highlight box when selecting accessible nodes',
  'hi': 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि'},
 {'en': 'Highlight border color',
  'hi': 'सीमांत (बोर्डर) के रंग को हाइलाइट करें'},
 {'en': 'The color and opacity of the highlight border.',
  'hi': 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। '},
 {'en': 'Highlight fill color', 'hi': 

In [25]:
for ex in  dataset["train"]["translation"][0:30]:   # first 31 sentences
  print(ex["en"])   # fetching english sentences

Accerciser Accessibility Explorer
The default plugin layout for the bottom panel
The default plugin layout for the top panel
A list of plugins that are disabled by default
Highlight duration
The duration of the highlight box when selecting accessible nodes
Highlight border color
The color and opacity of the highlight border.
Highlight fill color
The color and opacity of the highlight fill.
API Browser
Browse the various methods of the current accessible
Hide private attributes
Method
Property
Value
IPython Console
Interactive console for manipulating currently selected accessible
Event monitor
_ Monitor Events
C _ lear Selection
Everything
Selected application
Selected accessible
Source
Event Monitor
Shows events as they occur from selected types and sources
Highlight last event entry
Start / stop event recording


In [26]:
for ex in  dataset["train"]["translation"][0:30]:   # first 31 sentences
  print(ex["hi"])   # fetching hindi sentences

एक्सेर्साइसर पहुंचनीयता अन्वेषक
निचले पटल के लिए डिफोल्ट प्लग-इन खाका
ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
अवधि को हाइलाइट रकें
पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि
सीमांत (बोर्डर) के रंग को हाइलाइट करें
हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। 
भराई के रंग को हाइलाइट करें
हाइलाइट किया गया भराई का रंग और पारदर्शिता। 
एपीआई विचरक
इस समय जिसे प्राप्त किया गया हो, उसकी विभिन्न विधियों (मेथड) में विचरण करें
निजी गुणों को छिपाएं
विधि
गुणधर्म
मान
आईपाइथन कन्सोल
इस समय चुने गए एक्सेसेबेल से काम लेने के लिए अंतर्क्रियात्मक कन्सोल
घटना मानिटर
घटनाओं को मानिटर करें (_ M) 
चुनाव को हटाएं (C _) 
सभी
चुने गए अनुप्रयोग
चुने गए एक्सेसेबेल
स्रोत
घटना मानिटर
चुने गए प्रकारों और स्रोतों से घटनाएं जैसे-जैसे घटित होती हैं, उन्हें दर्शाता है
अंतिम प्रविष्ट घटना को हाइलाइट करो
घटना रेकोर्डिंग शुरू करो/रोको


In [31]:
max_input_length = 128
max_target_length = 128

source_lang="en"
target_lang="hi"

def process_function(sentence):

  inputs= [ex[source_lang] for ex in sentence["translation"]]
  targets= [ex[target_lang] for ex in sentence["translation"]]

  model_inputs = tokenizer(inputs,max_length=max_input_length,truncation=True)

  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets,max_length=max_target_length,truncation=True)


  model_inputs["labels"]=labels["input_ids"]
  return model_inputs


In [32]:
tokenized_data = dataset.map(process_function, batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [33]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2507
    })
})

In [34]:
model

'Helsinki-NLP/opus-mt-en-hi'

In [35]:
# i am loading pre trained model
model=TFAutoModelForSeq2SeqLM.from_pretrained(model)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [36]:
batch_size=32
learning_rate=0.001
weight_decay=0.01
num_train_epochs=1

In [37]:
data_collator=DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [38]:
train_dataset=model.prepare_tf_dataset(
    tokenized_data["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator

)

In [39]:
validation_dataset=model.prepare_tf_dataset(
    tokenized_data["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,

)

In [40]:
optimizer=AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)

In [41]:
model.compile(optimizer=optimizer)

In [42]:
#this is my finetuing
model.fit(train_dataset,validation_data=validation_dataset,epochs=num_train_epochs)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
78/78 [==============================] - 182s 821ms/step - loss: 4.5893 - val_loss: 4.8000


In [43]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


#model inferencing
https://huggingface.co/datasets/cfilt/iitb-english-hindi

In [44]:
input_text="i am learning coding. how about your coding"

In [45]:
tokenized=tokenizer([input_text],return_tensors='np')

In [46]:
tokenized

{'input_ids': array([[5556,  489, 3729, 7308,  232,  137,    3,  287,  195,   85, 7308,
         232,  137,    0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [47]:
out=model.generate(**tokenized, max_length=128)

In [48]:
out[0]

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([61949,   104,  1811,    57,   153,   254,    40,   522,   697,
         123,  1890,     6,   207,    11,  1374,    91,     6,   207,
          11,    65,  1374,    78,     0, 61949], dtype=int32)>

In [53]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0]))


<pad> मैं चर्चा कर रहा हूँ। आपके कवरिंग के बारे में विचार करने के बारे में क्या विचार किया</s> <pad>


In [ ]:
# hugging:
# openai
# gemini
# langcain
# llamaindex
# vectordatabase

# application
# RAG

# 1 month


# opesource
# hugging: gemma llama mistral claude
# fietiuning
# PEFT
# LORA
# QLORA
# Dissliation
# quantization

# end to end:
# deployement
# llmops
# mlops
# aws,azure

# difffion
# tools